# Projeto Análise de Textos de Fontes Desestruturas e Web

### Importando Bibliotecas:

In [1]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

### Configuração:

In [2]:
my_phone_number="+5511991982436"
CHROME_PROFILE_PATH = "user-data-dir=C:\\Users\\gabri\\AppData\\Local\\Google\\Chrome\\User Data\\WebScrapping\\Projeto"

In [3]:
options = webdriver.ChromeOptions()
options.add_argument(CHROME_PROFILE_PATH)

wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

### Setup de Localização de Entrega:
Executar somente uma vez, a localização será guardada nos cookies da sessão.

In [4]:
# wd.get("https://amazon.com")

# import time
# time.sleep(5)

# wd.find_element(By.ID, "nav-global-location-popover-link").click()
# time.sleep(2)
# postcode_field = wd.find_element(By.ID, "GLUXZipUpdateInput")
# postcode_field.send_keys("80021")
# time.sleep(2)
# wd.find_element(By.ID, "GLUXZipUpdate").click()
# close_element = wd.find_element(By.XPATH, '//*[@id="a-popover-1"]/div/div[2]/span/span/span/button')
# close_element.click()

### Web Scrapping da Amazon:

In [5]:
wd.get("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(wd.page_source)

results = soup.findAll("div", {"data-component-type": "s-search-result"})
for result in results:
    title = result.find("span", {"class": "a-text-normal"})
    sponsored = result.find("span", {"class": "a-color-base"}, string="Sponsored")
    price = result.find("span", {"class": "a-price-whole"})
    if not sponsored and price:
        product = title.text
        price = price.text
        print(f'* {price} - {product}')


* 717. - Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4.0 Graphics Card Titanium and Black (Renewed)
* 699. - EVGA GeForce RTX 3080 Ti FTW3 Ultra Gaming, 12G-P5-3967-KR, 12GB GDDR6X, iCX3 Technology, ARGB LED, Metal Backplate (Renewed)
* 684. - MSI Gaming GeForce RTX 3080 Ti Ventus 3X 12G OC - 12GB GDDR6X Graphic Card for PC Gaming, 320-Bit HDMI/DP, NVIDIA GPU, Tri-Frozr 2 Cooling, Ampere Architecture, Computer Video Graphics Card (Renewed)
* 749. - ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC Edition Graphics Card (PCIe 4.0, 12GB GDDR6X, HDMI 2.1, DisplayPort 1.4a, Dual Ball Fan Bearings, Military-Grade Certification, GPU Tweak II) (Renewed)
* 689. - MSI Gaming GeForce RTX 3080 Ti Gaming X Trio 12G Computer Gaming Graphics Card 12GB GDRR6X 320-Bit HDMI/DP Nvlink Torx Fan 3 Ampere Architecture OC, NVIDIA GPU Video Cards for Gaming PC (Renewed)
* 979. - Geforce RTX 3080 Ti 12GB GDDR6X PCI Express 4.0 Graphics Card Titanium and Black
* 799. - EVGA GeForce RTX 3080 Ti XC3 Ultra Gaming, 12

In [7]:
def getPandaDfForSearchResults(searchResultsPage):
    rows = []
    for result in searchResultsPage:
        title = result.find("span", {"class": "a-text-normal"})
        sponsored = result.find("span", {"class": "a-color-base"}, string="Sponsored")
        price = result.find("span", {"class": "a-price-whole"})
        url = result.find("a", {"class": "a-link-normal"})
        if price:
            row= [title.text, bool(sponsored), price.text, "https://amazon.com/" + url['href']]
            rows.append(row)
    
    df = pd.DataFrame.from_records(rows, columns=["Title", "Sponsored", "Price", "URL"])
    return df

In [8]:
import pandas as pd

wd.get("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")

nextPageCssClasses=[]
nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")


df_all_search_results = pd.DataFrame(columns=["Title", "Sponsored", "Price"])
while not "s-pagination-disabled" in nextPageCssClasses:
    soup = BeautifulSoup(wd.page_source)
    searchResultsPage = soup.findAll("div", {"data-component-type": "s-search-result"})
    df = getPandaDfForSearchResults(searchResultsPage)
    df_all_search_results = pd.concat([df_all_search_results, df])
                              
    # go to next page
    nextPageLink.click()
    nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")
    nextPageCssClasses = nextPageLink.get_attribute("class").split()
    time.sleep(2)

if "s-pagination-disabled" in nextPageCssClasses:
    print("Reached last page of search results")

df_all_search_results    
                            

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".s-pagination-next"}
  (Session info: chrome=125.0.6422.77); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00DFB793+45827]
	(No symbol) [0x00D8DB74]
	(No symbol) [0x00C8150F]
	(No symbol) [0x00CC20BC]
	(No symbol) [0x00CC216B]
	(No symbol) [0x00CFE0F2]
	(No symbol) [0x00CE2E44]
	(No symbol) [0x00CFC034]
	(No symbol) [0x00CE2B96]
	(No symbol) [0x00CB6998]
	(No symbol) [0x00CB751D]
	GetHandleVerifier [0x010B43C3+2899763]
	GetHandleVerifier [0x011077ED+3240797]
	GetHandleVerifier [0x00E81264+593364]
	GetHandleVerifier [0x00E8818C+621820]
	(No symbol) [0x00D96F54]
	(No symbol) [0x00D93658]
	(No symbol) [0x00D937F7]
	(No symbol) [0x00D858AE]
	BaseThreadInitThunk [0x75627BA9+25]
	RtlInitializeExceptionChain [0x773BBE3B+107]
	RtlClearBits [0x773BBDBF+

In [ ]:
def getSearchResultsForPage(url):
    wd.get(url)
    nextPageCssClasses=[]
    nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")


    df_all_search_results = pd.DataFrame(columns=["Title", "Sponsored", "Price"])
    while not "s-pagination-disabled" in nextPageCssClasses:
        soup = BeautifulSoup(wd.page_source)
        searchResultsPage = soup.findAll("div", {"data-component-type": "s-search-result"})
        df = getPandaDfForSearchResults(searchResultsPage)
        df_all_search_results = pd.concat([df_all_search_results, df])

        # go to next page
        nextPageLink.click()
        nextPageLink = wd.find_element(By.CLASS_NAME, "s-pagination-next")
        nextPageCssClasses = nextPageLink.get_attribute("class").split()
        time.sleep(2)

    if "s-pagination-disabled" in nextPageCssClasses:
        print("Reached last page of search results")

    df_all_search_results['Price'] = df_all_search_results['Price'].str.replace(",","")
    df_all_search_results['Price'] = df_all_search_results['Price'].astype(float)
    return df_all_search_results    
                            

In [ ]:
# getSearchResultsForPage("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")

In [ ]:
import pywhatkit as kit
def sendWhatsAppMessage(df):
    for index, graphicsCardRow in df.iterrows():
        print(graphicsCardRow.URL)
        wd.get(graphicsCardRow.URL)
        screenshotFilepath="C:\\Users\\gabri\\Downloads\\screenshot_{index}.png"
        wd.save_screenshot(screenshotFilepath)
        kit.sendwhats_image(my_phone_number, screenshotFilepath, graphicsCardRow.URL, tab_close=True)


In [ ]:
while (True):
    df = getSearchResultsForPage("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")
    
    df_query = df[(df.Title.str.contains("3080 Ti")) & (df.Price <= 2200)]
    
    if (not df_query.empty):
        print("Found products that I want to buy")
        sendWhatsAppMessage(df_query)
        print(df_query)
        break
    else:
        time.sleep(60)
    

### Análise de Dados

---